**Part 1: Data Acquisition and Extraction
Task 1.1 Crawl and Save**

In the notebook cell, I logged into the LyricsDB site using the provided credentials and the site’s IP address. To confirm successful login, I checked for the presence of "LyricsDB" in the HTML of the response.

The lyrics were retrieved by sending programmatic HTTP requests to the site’s API using a session authenticated with the login credentials. Each page of songs was requested via a POST call, which returned JSON data.

Finally, the songs retrieved from all pages were saved into a CSV file in the project’s dedicated folder, Songs, named crawled_songs.csv. Since I needed to use all songs for NLP analysis, I scraped all available songs from the site.

In [1]:
import requests
import csv
import os

def scrape_songs(max_pages=120):
    """
    Scrape songs from LyricsDB and save them to a CSV file.

    Args:
        max_pages (int): Maximum number of pages to scrape.
    """

    # ------------------------------
    # User credentials
    # ------------------------------
    username = 'ivan.borg.i84578@mcast.edu.mt'
    password = '0210JjtzUvKc'
    base_url = 'http://23.94.19.185'

    # ------------------------------
    # Start session
    # ------------------------------
    session = requests.Session()
    session.auth = (username, password)

    # ------------------------------
    # Verify login
    # ------------------------------
    home_resp = session.get(base_url)
    if "LyricsDB" not in home_resp.text:
        print("Login failed — LyricsDB not found.")
        return

    print("Logged in successfully — LyricsDB detected!")

    # ------------------------------
    # Scrape pages
    # ------------------------------
    all_songs_url = f'{base_url}/all_songs.php'
    all_songs = []

    for page in range(1, max_pages + 1):
        payload = {"page": page}
        resp = session.post(all_songs_url, json=payload)
        resp.raise_for_status()

        data = resp.json()
        songs = data.get('songs', [])
        total = data.get('total', 0)

        if not songs:
            print(f"No songs found on page {page}. Stopping...")
            break

        all_songs.extend(songs)
        print(f"Scraping page {page}: {len(all_songs)} songs collected.")

        if total and len(all_songs) >= total:
            print("All songs scraped successfully.")
            break

    # ------------------------------
    # Save to CSV (once after all pages)
    # ------------------------------
    csv_filename = "Songs/crawled_songs.csv"
    if all_songs:
        headers = all_songs[0].keys()
        os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

        with open(csv_filename, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=headers)
            writer.writeheader()
            writer.writerows(all_songs)

        print(f"Saved {len(all_songs)} songs to {csv_filename}")
    else:
        print("No songs found to save")

    return None

    # ------------------------------
    # execute function
    # ------------------------------
scrape_songs(max_pages=100)


Logged in successfully — LyricsDB detected!
Scraping page 1: 20 songs collected.
Scraping page 2: 40 songs collected.
Scraping page 3: 60 songs collected.
Scraping page 4: 80 songs collected.
Scraping page 5: 100 songs collected.
Scraping page 6: 120 songs collected.
Scraping page 7: 140 songs collected.
Scraping page 8: 160 songs collected.
Scraping page 9: 180 songs collected.
Scraping page 10: 200 songs collected.
Scraping page 11: 220 songs collected.
Scraping page 12: 240 songs collected.
Scraping page 13: 260 songs collected.
Scraping page 14: 280 songs collected.
Scraping page 15: 300 songs collected.
Scraping page 16: 320 songs collected.
Scraping page 17: 340 songs collected.
Scraping page 18: 360 songs collected.
Scraping page 19: 380 songs collected.
Scraping page 20: 400 songs collected.
Scraping page 21: 420 songs collected.
Scraping page 22: 440 songs collected.
Scraping page 23: 460 songs collected.
Scraping page 24: 480 songs collected.
Scraping page 25: 500 songs colle

In the next cell, since the retrieved songs were still contained in the all_songs csv, I opted to store them in an SQL Express database called TextMiningHA for more convenient data manipulation. Dynamically, I created a table called songs and, from all the scraped information, used only the columns song_id, track_name, artists, lyrics, and genre. All extracted data were saved into their respective columns in the database.

Additionally, I added three more columns for NLP processing: cleanGenre, tokenized, and cleanTokens. All processed data generated in the next project tasks will be saved into their respective columns accordingly.

In [2]:
import pyodbc
import pandas as pd

try:
    # Connect to SQL Server
    conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r'SERVER=IVAN_PC\SQLEXPRESS;'
        r'DATABASE=TextMiningHA;'
        r'Trusted_Connection=yes;'
    )
    cursor = conn.cursor()

    # Create the "songs" table if it doesn't already exist
    cursor.execute('''
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='songs' AND xtype='U')
    CREATE TABLE songs (
        song_id NVARCHAR(35) PRIMARY KEY,  -- unique song identifier
        name NVARCHAR(100),                -- track name
        artist NVARCHAR(255),              -- artist name(s)
        lyrics NVARCHAR(MAX),              -- lyrics text
        genre NVARCHAR(MAX),               -- original genre
        cleanGenre NVARCHAR(MAX),          -- cleaned genre for NLP
        tokenised NVARCHAR(MAX),           -- tokenized lyrics
        cleanTokens NVARCHAR(MAX)          -- cleaned tokens for NLP
    )
    ''')
    conn.commit()
    print("Connection successful and table ready.")

except pyodbc.Error as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

# Counter for inserted rows
rows_added = 0

# Load songs from CSV file in project root
try:
    df = pd.read_csv("Songs/crawled_songs.csv")

    for _, row in df.iterrows():
        song_id = str(row.get('song_id')) if pd.notna(row.get('song_id')) else None
        name = str(row.get('track_name')) if pd.notna(row.get('track_name')) else None
        artist = str(row.get('artists')) if pd.notna(row.get('artists')) else None
        lyrics = str(row.get('lyrics')) if pd.notna(row.get('lyrics')) else None
        genre = str(row.get('genre')) if pd.notna(row.get('genre')) else None

        # Check if the song already exists in the database
        cursor.execute("SELECT 1 FROM songs WHERE song_id = ?", (song_id,))
        if cursor.fetchone() is None:
            cursor.execute(
                "INSERT INTO songs (song_id, name, artist, lyrics, genre) VALUES (?, ?, ?, ?, ?)",
                (song_id, name, artist, lyrics, genre)
            )
            rows_added += 1

    # Commit after inserting all rows
    conn.commit()

except Exception as e:
    print(f"Error inserting songs: {e}")
finally:
    conn.close()
    print(f"Inserted {rows_added} new songs.\r\nConnection closed.")


Connection successful and table ready.
Inserted 0 new songs.
Connection closed.


 In the next cell, I included commands to connect to the SQL database, as well as operations for managing the data. This included deleting table or columns when needed, and fetching specific songs by applying filters. Mainly used as testing tool.

In [3]:
from sqlalchemy import create_engine

# Create the SQLAlchemy engine
try:
    engine = create_engine("mssql+pyodbc://IVAN_PC\\SQLEXPRESS/TextMiningHA?driver=ODBC+Driver+17+for+SQL+Server")
    print("Connected to SQL Server!")


except Exception as e:
    print("Connection failed:")
    print(e)

from sqlalchemy import text

try:
    conn = engine.connect()
    trans = conn.begin()  # start a transaction manually
    try:
        #conn.execute(text("UPDATE songs SET cleanTokens = NULL"))       #clear cleanTokens Column
        #conn.execute(text("DELETE FROM songs WHERE genre LIKE '%K%'"))  #delete rows where genre contains char K
        #conn.execute(text("DROP TABLE IF EXISTS songs"))                #delete entire table
        trans.commit()  # explicitly commit
    except Exception:
        trans.rollback()  # roll back if something fails inside
        raise
    finally:
        conn.close()  # make sure the connection is closed

except Exception as e:
    print(e)

Connected to SQL Server!
